In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

In [5]:
# Carregando os dados necessários
bibles_lcm_file = '../dataset/bibles_lcm.csv'
bibles_90_lcm_file = '../dataset/bibles_90_lcm.csv'

df_lcm = pd.read_csv(bibles_lcm_file)
df_90_lcm = pd.read_csv(bibles_90_lcm_file)

# Inspecionar os primeiros registros dos dataframes
print("Colunas de df_90_lcm:")
print(df_90_lcm.columns)
print("\nPrimeiras linhas de df_90_lcm:")
print(df_90_lcm.head())

print("\nColunas de df_lcm:")
print(df_lcm.columns)
print("\nPrimeiras linhas de df_lcm:")
print(df_lcm.head())

Colunas de df_90_lcm:
Index(['book', 'chapter', 'language', 'language_wals_code', 'verse_number',
       'text'],
      dtype='object')

Primeiras linhas de df_90_lcm:
  book  chapter       language language_wals_code  verse_number  \
0  ROM        1  ANCIENT_GREEK              [grc]             1   
1  ROM        1        APALAÍ                apl             1   
2  ROM        1        APURINÃ                apu             1   
3  ROM        1      ASHENINKA                cax             1   
4  ROM        1        ENGLISH                eng             1   

                                                text  
0  Παῦλος δοῦλος Χριστοῦ Ἰησοῦ, κλητὸς ἀπόστολος,...  
1  Ynymerohpyry moro onenerykõme Roma põkomo. Ywy...  
2  Nota Paoro iokanatsopata hĩtemoni. Xesosi Kris...  
3  Naaka Pablo osankenatzimirori, rimperatanewo J...  
4  From Paul, a slave of Christ Jesus, called to ...  

Colunas de df_lcm:
Index(['book', 'chapter', 'language', 'language_wals_code', 'verse_number',
   